## Lab 9: Digit Orientation Prediction with PyTorch
In this lab you will implement a version of image orientation detector in PyTorch. You will explore image directory based data loaders, building models using PyTorch pre-defined layers, implementing mini-batch learning using data loaders, and using input transformations for data augmentation.

### Question 1: Data Loading
The data for this lab are contained in raw image files instead of pre-loaded into a data array. They can be loaded using a PyTorch data loader. The data loader applies a transform to turn the raw images it loads into vectors. The data loader also takes care of shuffling and batching data. Below we show how to create a data loader for the training set. Add code to load the validation and test sets.   

In [60]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Grayscale(), #convert to grayscale
    transforms.ToTensor(), #convert to a tensor
    transforms.Lambda(torch.flatten) #flatten to vectors
])

batch_size    = 1024
train_dataset = ImageFolder(root='data/train', transform=transform)
train_loader  = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=0, pin_memory=True)

Once the data loader is defined, you can sample a batch of random data vectors and labels from it to verify the data loading and transformations are correct.

In [ ]:
X, Y = next(iter(train_loader))
print(X.shape)
print(Y.shape)

### Question 2: Data Visualization
Use matplotlib to visualize the first five training, validation and test vectors returned by each data loader. You will need to transform the images back into 28x28 images and then display them. Show a title for each image specifying the image's label as both a string and numerical value. The string labels for each class are contained in the dataset objects.

In [ ]:
import matplotlib.pyplot as plt

X, Y = next(iter(train_loader))



### Question 3: Model Construction
For a model, you will implement probabilistic logistic regression as a PyTorch neural network module. A basic module requires specifying just two methods. __init__() and forward(). 

The init method initializes the model object. For this model, we include a single argument D, the length of the input data vector. You should implement the model using PyTorch's existing linear layer type nn.Linear. See the PyTorch API documents for information on how to define and use this layer: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html.

The forward method method should compute the probability that an input vector belongs to class 1. This requires the application of the linear layer, followed by the sigmoid/logistic function. See the documentation for the sigmoid function here: https://pytorch.org/docs/stable/generated/torch.nn.functional.sigmoid.html. The implementation needs to be vectorized so it computes the required probability for each row of the input tensor X.

In [63]:
import torch.nn as nn
import torch.nn.functional as F

class plr(nn.Module):
    def __init__(self,D):
        super(plr, self).__init__()
        #Add code here

    def forward(self, X):
        #Ad code here
        pass

To verify that your implementation runs without error, try the following code. This code creates a model object, then runs the forward function to compute output class probabilities. The probabilities will be computed using the default randomly initialized model parameters used by the pre-defined fully connected layer. If the code fails to run, it has bugs and needs more work. 

In [ ]:
X, Y  = next(iter(train_loader))
model = plr(784)
P     = model.forward(X)
P[:10]

### Question 4: Model Training
In this question you will complete the specification of code needed to optimize the model parameters. The missing components are the execution of the forward pass to produce output probabilities and predictions, and the computation of the negative log likelihood. You can either compute the negative log likelihood from scratch, or investigate the use of PyTorch's built in loss function binary_cross_entropy() documented here: https://pytorch.org/docs/stable/generated/torch.nn.functional.binary_cross_entropy.html. Once you have the missing components implemented, execute the code to verify that the implementation runs and the loss and training error are decreasing.

In [ ]:
from torch import optim

device = "cuda" if torch.cuda.is_available() else "cpu" #Set device to gpu or cpu

model      = plr(784) 
lr         = 0.001
num_epochs = 100
train_loss = []
train_err  = []

optimizer  = optim.Adam(model.parameters(), lr=lr,weight_decay=1e-1)

# Start loop
for epoch in range(num_epochs): #(loop for every epoch)

    model.train()    # training mode
    running_loss = 0.0   
    running_error = 0.0 

    for i, (X, Y) in enumerate(train_loader): # load a batch data of images

        #Move batch to device if needed
        X = X.to(device)
        Y = Y.to(device) 

        optimizer.zero_grad() #Zero the gradient

        P    =           #Compute predicted probabilities
        P    =  torch.clip(P,0.01,0.99).flatten() #Clip the probabilities to avoid log(0)
        Yhat =           #Compute predictions

        loss =           #Compute the nll loss

        loss.backward()       #Compute the gradient of the loss
        optimizer.step()      #Take a step

        # update running loss and error
        running_loss  += loss.item() * X.shape[0]
        running_error += torch.sum(Yhat.flatten() != Y.flatten()).item()

    #Compute loss for the epoch
    epoch_loss  = running_loss /  len(train_loader.dataset)
    epoch_error = running_error / len(train_loader.dataset) 

    # Append result
    train_loss.append(epoch_loss)
    train_err.append(epoch_error)

    # Print progress
    print('[Train #{}] Loss: {:.8f} Err: {:.4f}'.format(epoch+1, epoch_loss, epoch_error))

### Question 5: Evaluation 
Now use the validation and test set data loaders to evaluate the prediction error rate of the learned model. Since both the data loaders use batching, you will need to loop over all of the batches to compute the test and validation error rates. Add your code below.

### Question 6: Data Augmentation
One way to make an image classification model more robust is to apply augmentations to the training data set during learning such as adding pixel noise, random rotations, random crops, image blur, etc. These data augmentations need to be selected such that you would expect the label of the instance to be preserved. Try experimenting with adding additional image transforms to the training data set loader. The transforms implemented in the TorchVision package are documented here: https://pytorch.org/vision/0.20/transforms.html. Such data augmentations are applied on the fly in the data loader each time a batch is returned. Make sure you *do not* add these augmentations to the validation and test data loaders. See if you can improve the generalization performance of the model using data augmentations. 